# 7j – Multi‑criteria decision analysis (MCDA)

Perform **TOPSIS** multi‑criteria ranking for each spoke, using three illustrative criteria (distance, hub capacity, current wait time) with adjustable weights.

In [ ]:
import math

import pandas as pd, numpy as np
from pathlib import Path

DATA_DIR = Path('.')
ACUTE_CSV = DATA_DIR / 'NHS_SW_Acute_Hospitals_enriched.csv'
CDC_CSV   = DATA_DIR / 'NHS_SW_Community_Diagnostic_Centres_enriched.csv'
CH_CSV    = DATA_DIR / 'NHS_SW_Community_Hospitals_enriched.csv'

acute = pd.read_csv(ACUTE_CSV)
spokes = pd.concat([pd.read_csv(CDC_CSV), pd.read_csv(CH_CSV)], ignore_index=True)

# Example criteria weights
weights = {
    'distance': 0.5,
    'capacity': 0.3,
    'wait_time': 0.2
}

# Simple TOPSIS implementation ------------------------------------------
R_EARTH = 6371
def haversine(lat1, lon1, lat2, lon2):
    φ1, λ1, φ2, λ2 = map(math.radians, (lat1, lon1, lat2, lon2))
    dφ, dλ = φ2 - φ1, λ2 - λ1
    a = math.sin(dφ/2)**2 + math.cos(φ1)*math.cos(φ2)*math.sin(dλ/2)**2
    return 2 * R_EARTH * math.atan2(math.sqrt(a), math.sqrt(1-a))

results = []
for _, spk in spokes.iterrows():
    scores = []
    for _, hub in acute.iterrows():
        distance = haversine(spk.latitude, spk.longitude, hub.latitude, hub.longitude)
        capacity = hub.get('capacity', 1)
        wait = hub.get('wait_time', 1)
        scores.append({'hub': hub.Name, 'distance': distance,
                       'capacity': capacity, 'wait_time': wait})
    df = pd.DataFrame(scores)
    # Normalise
    for col in ['distance', 'capacity', 'wait_time']:
        df[col] = (df[col] - df[col].min()) / (df[col].max() - df[col].min() + 1e-9)
    # Ideal / nadir
    ideal = df[['distance', 'capacity', 'wait_time']].max()
    nadir = df[['distance', 'capacity', 'wait_time']].min()
    # Separation
    sep_pos = np.sqrt(((df[['distance','capacity','wait_time']] - ideal)**2 * list(weights.values())).sum(axis=1))
    sep_neg = np.sqrt(((df[['distance','capacity','wait_time']] - nadir)**2 * list(weights.values())).sum(axis=1))
    df['topsis_score'] = sep_neg / (sep_pos + sep_neg + 1e-9)
    best_hub = df.loc[df.topsis_score.idxmax(), 'hub']
    results.append({'spoke': spk.Name, 'best_hub': best_hub})
pd.DataFrame(results).head()